In [62]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient
from pdfminer.high_level import extract_text

In [16]:
load_dotenv()
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")
model: str = "text-embedding-ada-002" 
credential = AzureKeyCredential(key)
print(service_endpoint)

https://search-crew512upskill.search.windows.net


In [10]:
# pdf to strings
# Extract text from a pdf.
files = [os.path.join(path, name) for path, _, files in os.walk('rawData') for name in files]

data = list()
for index, file in enumerate(files):
  data.append({
    "id": index,
    "title": file,
    "content": extract_text(file),
    "category": "hr docs"
  })

with open('hr_data.json', 'w') as file:
    json.dump(data, file)

In [55]:
# Generate Document Embeddings using OpenAI Ada 002
# Read the text-sample.json
with open('hr_data.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.embeddings.create(
        input=text,model="text-embedding-ada-002"
    )
    embeddings = response.data[0].embedding
    return embeddings

# Generate embeddings for title and content fields
for item in input_data:
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings

# Output embeddings to docVectors.json file
with open("docVectors.json", "w") as f:
    json.dump(input_data, f)

In [64]:
#from azure.search.documents.models import 
from azure.search.documents.indexes.models import SimpleField, SearchFieldDataType, SearchableField, SearchField, VectorSearch, VectorSearchProfile, VectorSearchAlgorithmConfiguration, VectorSearchAlgorithmKind, HnswParameters, ExhaustiveKnnParameters, SemanticPrioritizedFields, SemanticConfiguration, SemanticField, SearchIndex, SemanticSearch, ExhaustiveKnnAlgorithmConfiguration, HnswAlgorithmConfiguration

# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile="myHnswProfile"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric="cosine"
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric="cosine"
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm="myHnsw",
            vectorizer="myOpenAI",
            algorithm_configuration_name="myHnsw"
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm="myExhaustiveKnn",
            vectorizer="myOpenAI",
            algorithm_configuration_name="myExhaustiveKnn"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="myOpenAI",
            kind="azureOpenAI",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=os.getenv("AZURE_OPENAI_ENDPOINT"),
                deployment_id=model,
                api_key=os.getenv("AZURE_OPENAI_API_KEY")
            )
    )  
]  

)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="category")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)

searchClient = SearchIndexClient(
  endpoint=os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT"),
  credential=AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")))

result = searchClient.create_index(index)

# result = index_client.create_or_update_index(index)

print(f' {result.name} created')

vector_search_profile is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
vector_search_profile is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
algorithm is not a known attribute of class <class 'azure.search.documents.indexes._generated.models._models_py3.VectorSearchProfile'> and will be ignored
vectorizer is not a known attribute of class <class 'azure.search.documents.indexes._generated.models._models_py3.VectorSearchProfile'> and will be ignored
algorithm is not a known attribute of class <class 'azure.search.documents.indexes._generated.models._models_py3.VectorSearchProfile'> and will be ignored
vectorizer is not a known attribute of class <class 'azure.search.documents.indexes._generated.models._models_py3.VectorSearchProfile'> and will be ignored


NameError: name 'AzureOpenAIVectorizer' is not defined

In [ ]:
# Upload some documents to the index  
with open('docVectors.json', 'r') as file:  
    documents = json.load(file)  
  
# Use SearchIndexingBufferedSender to upload the documents in batches optimized for indexing  
with SearchIndexingBufferedSender(  
    endpoint=service_endpoint,  
    index_name=index_name,  
    credential=credential,  
) as batch_client:  
    # Add upload actions for all documents  
    batch_client.upload_documents(documents=documents)  
print(f"Uploaded {len(documents)} documents in total") 